In [51]:
import pyspark
sc = pyspark.SparkContext('local[*]')

# Spark SQLContext

## Documentation: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html

Initilizing SQLContext

In [52]:
!rm -rf metastore_db/
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

## Creating user-defined functions

To use RDD in SQLContext, RDD lines have to be converted into a Row format

In [53]:
import re
from pyspark.sql import Row
# Read from CSV
def load_csv(line):
    return re.split("\"?,\"?", line) # "?,"?
        
def readInt(x):
    try:
        return int(x)
    except:
        return x

def parseElement(e):
    return Row(year=readInt(e[0]),
    month=readInt(e[1]),
    day=readInt(e[2]),
    dep_time=readInt(e[3]),
    dep_delay=readInt(e[4]),
    arr_time=readInt(e[5]),
    arr_delay=readInt(e[6]),
    cancelled=e[7],
    carrier=e[8],
    tailnum=e[9],
    flight=readInt(e[10]),
    origin=e[11],
    dest=e[12],
    air_time=readInt(e[13]),
    distance=readInt(e[14]),
    hour=readInt(e[15]),
    min=readInt(e[16])
    )

In [54]:
!wget https://dsr-data.s3.amazonaws.com/flights/flights14.csv

--2018-02-16 10:14:10--  https://dsr-data.s3.amazonaws.com/flights/flights14.csv
Resolving dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)... 52.219.73.46
Connecting to dsr-data.s3.amazonaws.com (dsr-data.s3.amazonaws.com)|52.219.73.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16150465 (15M) [text/csv]
Saving to: ‘flights14.csv.1’

flights14.csv.1     100%[=====================>]  15.40M  3.89MB/s   in 4.7s   

2018-02-16 10:14:15 (3.31 MB/s) - ‘flights14.csv.1’ saved [16150465/16150465]



In [55]:
flights = sc.textFile("flights14.csv").map(load_csv).filter(lambda e: not e[0] == "year").map(parseElement).cache()

Now the schema and the RDD have to be registered with the sqlContext:

In [56]:
flightsDF = sqlContext.createDataFrame(flights)

In [57]:
flightsDF.createOrReplaceTempView("flight")

In [58]:
sqlContext.sql("select * from flight where dest = 'LAX'").take(5)

[Row(air_time=359, arr_delay=13, arr_time=1238, cancelled='0', carrier='AA', day=1, dep_delay=14, dep_time=914, dest='LAX', distance=2475, flight=1, hour=9, min=14, month=1, origin='JFK', tailnum='N338AA', year=2014),
 Row(air_time=363, arr_delay=13, arr_time=1523, cancelled='0', carrier='AA', day=1, dep_delay=-3, dep_time=1157, dest='LAX', distance=2475, flight=3, hour=11, min=57, month=1, origin='JFK', tailnum='N335AA', year=2014),
 Row(air_time=351, arr_delay=9, arr_time=2224, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1902, dest='LAX', distance=2475, flight=21, hour=19, min=2, month=1, origin='JFK', tailnum='N327AA', year=2014),
 Row(air_time=350, arr_delay=1, arr_time=1706, cancelled='0', carrier='AA', day=1, dep_delay=2, dep_time=1347, dest='LAX', distance=2475, flight=117, hour=13, min=47, month=1, origin='JFK', tailnum='N319AA', year=2014),
 Row(air_time=339, arr_delay=0, arr_time=2145, cancelled='0', carrier='AA', day=1, dep_delay=4, dep_time=1824, dest='LAX', d

In [59]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").count()

2750

In [60]:
flightsDF.where("origin = 'JFK' AND dest = 'MIA'").limit(2).collect()

[Row(air_time=161, arr_delay=-17, arr_time=1828, cancelled='0', carrier='AA', day=1, dep_delay=-1, dep_time=1509, dest='MIA', distance=1089, flight=145, hour=15, min=9, month=1, origin='JFK', tailnum='N5FJAA', year=2014),
 Row(air_time=166, arr_delay=-8, arr_time=1227, cancelled='0', carrier='AA', day=1, dep_delay=7, dep_time=917, dest='MIA', distance=1089, flight=1085, hour=9, min=17, month=1, origin='JFK', tailnum='N5DWAA', year=2014)]

In [61]:
flightsDF.count()

253316

### Count the flights that departed early and arrived late

In [62]:
flightsDF.where("dep_delay < 0 and arr_delay > 0").count()

30239

### Find the flight with the longest arrival delay

In [63]:
import pyspark.sql.functions as F

In [64]:
flightsDF.sort('arr_delay', ascending=False).limit(1).collect()
# flightsDF.agg({'arr_delay':'max'}).collect()

[Row(air_time=200, arr_delay=1494, arr_time=1008, cancelled='0', carrier='AA', day=4, dep_delay=1498, dep_time=727, dest='DFW', distance=1372, flight=1381, hour=7, min=27, month=10, origin='EWR', tailnum='N4WJAA', year=2014)]

### Find the top 10 destinations ordered by the number of flights

In [65]:
flightsDF.groupby(['dest']) \
    .agg({'dest':'count'}) \
    .sort('count(dest)', ascending=False) \
    .limit(10) \
    .show()

+----+-----------+
|dest|count(dest)|
+----+-----------+
| LAX|      14434|
| ATL|      12808|
| SFO|      11907|
| MCO|      11709|
| BOS|      11609|
| ORD|      11589|
| MIA|       9928|
| CLT|       9624|
| FLL|       9471|
| DCA|       6748|
+----+-----------+



### Find top 10 destinations with the worst avg arrival delay, ignoring flights that arrived early

In [66]:
flightsDF.filter("arr_delay >= 0") \
    .groupby(['dest']) \
    .agg({'arr_delay': 'mean'}) \
    .sort('avg(arr_delay)', ascending=False) \
    .limit(10) \
    .show()

+----+------------------+
|dest|    avg(arr_delay)|
+----+------------------+
| EGE| 78.03174603174604|
| AVP|              67.0|
| CAK| 55.86141304347826|
| TUL|55.122950819672134|
| MSN| 53.32692307692308|
| BGR| 52.28813559322034|
| OKC|  51.3609022556391|
| IAD|51.200559049615656|
| JAC|51.142857142857146|
| TVC| 49.64705882352941|
+----+------------------+



### Take a sample of 1% of the flights and then calculate the average departure delay for that sample

In [67]:
flightsDF.sample(False, 0.1, seed=42) \
    .agg({'dep_delay': 'mean'}) \
    .show()

+------------------+
|    avg(dep_delay)|
+------------------+
|12.278466564332664|
+------------------+



### For all flights from JFK during June, show the average departure delay for each destination

In [68]:
flightsDF.filter("origin == 'JFK' and month== 6") \
    .groupby(['dest']) \
    .agg({'dep_delay': 'avg'}) \
    .show()

+----+-------------------+
|dest|     avg(dep_delay)|
+----+-------------------+
| PSE| 17.133333333333333|
| MSY|  10.96629213483146|
| BUR| 16.428571428571427|
| OAK|  22.75862068965517|
| DCA|  5.372781065088757|
| ORF| 10.678571428571429|
| SAV|  4.267857142857143|
| CMH| 11.862068965517242|
| HNL|-1.7586206896551724|
| SJC| 12.241379310344827|
| CVG| 17.571428571428573|
| BUF| 12.900826446280991|
| AUS| 10.537931034482758|
| SJU|  8.947368421052632|
| LGB| 10.172413793103448|
| SRQ| -2.533333333333333|
| CHS|                6.4|
| RSW| 14.913793103448276|
| BOS| 13.006864988558352|
| LAS|  9.663841807909604|
+----+-------------------+
only showing top 20 rows



### For every origin/dest pair, count the number of flights

In [69]:
flightsDF.groupby(['origin', 'dest']).count().show()

+------+----+-----+
|origin|dest|count|
+------+----+-----+
|   EWR| STT|  174|
|   JFK| ORD| 1265|
|   JFK| BTV| 1004|
|   JFK| SRQ|  370|
|   EWR| CMH|  763|
|   LGA| CHO|  329|
|   JFK| JAC|    1|
|   JFK| ORF|  282|
|   LGA| DSM|    1|
|   LGA| PWM|  131|
|   JFK| MIA| 2750|
|   EWR| TPA| 1833|
|   EWR| DAY|  566|
|   EWR| BWI|  185|
|   LGA| CVG|   78|
|   LGA| ROA|  125|
|   LGA| ORF| 1014|
|   EWR| SAV|  557|
|   EWR| JAX|  998|
|   LGA| BNA| 2600|
+------+----+-----+
only showing top 20 rows



In [73]:
flightsDF.describe().show()

+-------+------------------+-----------------+------------------+---------+-------+------------------+------------------+------------------+------+------------------+------------------+------------------+------------------+------------------+------+-------+------+
|summary|          air_time|        arr_delay|          arr_time|cancelled|carrier|               day|         dep_delay|          dep_time|  dest|          distance|            flight|              hour|               min|             month|origin|tailnum|  year|
+-------+------------------+-----------------+------------------+---------+-------+------------------+------------------+------------------+------+------------------+------------------+------------------+------------------+------------------+------+-------+------+
|  count|            253316|           253316|            253316|   253316| 253316|            253316|            253316|            253316|253316|            253316|            253316|            253316| 

In [50]:
sc.stop()